# Recommender Systems 1
### Pavan Singh - MSc Advanced Analytics - 2023
#### An Introduction to Recommender Systems in Python


        The objective of a RecSys is to recommend relevant items for users, based on their preference. Preference and relevance are subjective, and they are generally inferred by items users have consumed previously

<br/><br/>

***
As previously discussed in the README.md file, there are three main families of methods for RecSys, these are:

1. **Collaborative Filtering**: This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person `A` has the same opinion as a person `B` on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person.
    * Imagine that there is a website that sells books, and we have data on which books each user has purchased. We can use this data to build a recommendation system that suggests books to users based on the purchases of other similar users. 

<br/><br/>

2. **Content-Based Filtering**: This method uses only information about the description and attributes of the items users has previously consumed to model user's preferences. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past (or is examining in the present). In particular, various candidate items are compared with items previously rated by the user and the best-matching items are recommended.
    * Imagine that there is a website that sells books, and each book has a number of attributes, such as the author, the publisher, and the genre. We can use these attributes to build a recommendation system that suggests books to users based on their past preferences. So we, for example, look at the attributes of the books person `A` has purchased and recommend similar books to them.

<br/><br/>

3. **Hybrid methods**: Recent research has demonstrated that a hybrid approach, combining collaborative filtering and content-based filtering could be more effective than pure approaches in some cases. These methods can also be used to overcome some of the common problems in recommender systems such as cold start and the sparsity problem.
    * Imagine that there is a website that sells books, and each book has a number of attributes, such as the author, the publisher, and the genre. We also have data on which books each user has purchased. We can use this data to build a recommendation system that combines content-based and collaborative filtering techniques. So first we can use content-based filtering to recommend books to user `A` based on the attributes of the books they have purchased. Then we can also use collaborative filtering to recommend books to user `A` based on the purchases of other similar users. We can then combine recommendations using various methods to give one list.

***
# Goal

We will demonstrate how to implement **Collaborative Filtering**, **Content-Based Filtering** and **Hybrid methods** in Python, for the task of providing personalized recommendations to the users. 

***
# Data

We make use of a data  set available on Kaggle: [*Articles sharing and reading from CI&T DeskDrop*](https://www.kaggle.com/datasets/gspmoreira/articles-sharing-reading-from-cit-deskdrop?select=users_interactions.csv). The dataset contains logs of users interactions on shared articles for the purpose of content Recommender Systems.  

The data set is useful as it contains additional item attributes, which would allow the application of Content-Based filtering techniques or Hybrid approaches, as well as collaborative filtering methods. 

Key features:
- contains a real sample of 12 months logs (Mar. 2016 - Feb. 2017) from CI&T's Internal Communication platform (DeskDrop).
- It contains about 73k logged users interactions on more than 3k public articles shared in the platform.
- two csv files, `shared_articles.csv`, `users_interactions.csv`

***

In [2]:
# import libraries (collection of modules)

import numpy as np  
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


- **Numpy**: provides support for large, multi-dimensional arrays and matrices of numerical data, and for performing mathematical operations on these
- **Pandas**: provides tools for data manipulation and analysis
- **Scikit-learn**: provides tools for machine learning and statistical modeling
- **Matplotlib**: provides functions for creating visualizations of data, such as plots, histograms, and scatter plots

# Shared Articles Data Set (`shared_articles.csv`)

Contains information about the articles shared in the platform. Each article has its:
- sharing date (timestamp), 
- the original url, 
- title, 
- content in plain text, 
- the article's language (lang), which is either Portuguese: "pt" or English: "en" and 
- information about the user who shared the article (author).

In [7]:
# load data  - articles

articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en


In [10]:
# attributes in data set
articles_df.columns

Index(['timestamp', 'eventType', 'contentId', 'authorPersonId',
       'authorSessionId', 'authorUserAgent', 'authorRegion', 'authorCountry',
       'contentType', 'url', 'title', 'text', 'lang'],
      dtype='object')

Note that: `CONTENT SHARED` means that the article was shared in the platform and is available for users, whilst `CONTENT REMOVED` means that the article was removed from the platform and not available for further recommendation. For this setting, we shall only consider here the `CONTENT SHARED` event type, assuming (naively) that all articles were available during the whole one year period. 

# User Interactions (`user_interactions.csv`)

Contains logs of user interactions on shared articles. 

- It can be **joined to `articles_shared.csv` by `contentId` column**.

Note that there are different eventType values;

- `VIEW`: The user has opened the article.
- `LIKE`: The user has liked the article.
- `COMMENT CREATED`: The user created a comment in the article.
- `FOLLOW`: The user chose to be notified on any new comment in the article.
- `BOOKMARK`: The user has bookmarked the article for easy return in the future.

In [9]:
# load data  - interactions
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.head(6)


,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR


In [ ]:
# see attributes of data frame
interactions_df.columns

# Data Munging/Wrangling

We transform the interactions data from an erroneous or unusable form into something more useful for our use case - RecSys. Specifically, we associate the different interactions (eventTypes) with a weight or strength, assuming that, for example, a comment in an article indicates a higher interest by the user on the item than a like, or than a simple view.

In [15]:
# create this  dictionary
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

# apply it using lambda function on eventType series
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

In [16]:
# check new data
interactions_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


# User-Cold Start

Recommender systems have a problem known as **user cold-start**, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.


As such, we opt to only use users who have at least 5 interactions.

In [52]:
# get number  of interactions per user
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))

# filter to get only users with more than 5. Only keep users!
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


What we are doing:

- Grouping the rows in the interactions_df dataframe by personId and contentId, and counting the number of rows in each group using the size() function.

- Grouping the resulting counts by personId, and again using the `size()` function to count the number of rows in each group. This produces a series of counts, where each count represents the number of unique contentId values that a particular personId has interacted with.

- Counting the number of rows in the resulting series using the `len()` function and printing the result.

- Filtering the series to only include rows where the count is greater than or equal to 5, and resetting the index of the resulting dataframe.

- Selecting only the personId column from the resulting dataframe and storing it in a new dataframe called users_with_enough_interactions_df.

- Counting the number of rows in users_with_enough_interactions_df and printing the result.


In [62]:
# see how many total interactions we have
print('# of interactions: %d' % len(interactions_df))

# get interactions from only those users who have greater than 5
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


We make use of a join to merge the dataframes - so as to get users with at least 5 interactions. Specifically we are:

- Printing the length of the `interactions_df` dataframe using the `len()` function. This will print the total number of interactions in the dataframe.

- Merging the `interactions_df` dataframe with the `users_with_enough_interactions_df` dataframe using the `merge()` function. The how parameter specifies the type of join to use, in this case a "right" join. The `left_on` and `right_on` parameters specify the columns to use for the merge.

- Printing the length of the resulting dataframe using the `len()` function. This will print the number of interactions from the users who have interacted with at least 5 unique contentId values.

## Assess User Interest per Article

In Deskdrop (CI&T Deskdrop), users are allowed to view an article many times, and interact with them in different ways (eg. like or comment). Thus, to model the user interest on a given article, we aggregate all the interactions the user has performed in an item by a weighted sum of interaction type strength and apply a log transformation to smooth the distribution.

Essentially we want to gauge the user interest on a given article they have interacted with. We do this by: 

1. aggregate all the interactions the user has performed in an item by a weighted sum of interaction type strength, and then 
2. apply a log transformation (to smooth the distribution)

In [ ]:
# define log transformation function
def smooth_user_preference(x):
    return math.log(1+x, 2)
    

interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

**Note:** The `\` character is used at the end of the first line to indicate that the following line is a continuation of the current line of code. It works by telling the interpreter to ignore the newline character at the end of the line and treat the next line as if it were still part of the current line. This allows you to write code that is more readable and easier to maintain by breaking up long lines of code.

I could of easily said:

``` python
interactions_from_selected_users_df.groupby(['personId', 'contentId'])['eventStrength'].sum().apply(smooth_user_preference).reset_index()
```

But it's naturally more difficult to read.



# Evaluation Pre-Processing

Evaluation is important for machine learning projects, because it allows to compare objectivelly different algorithms and hyperparameter choices for models. TO prepare for evaluation of our models - i.e., ensuring that the trained model generalizes for data it was not trained on - we shall make use of a holdout dataset. Here, a random data sample (20% in this case) are kept aside in the training process, and exclusively used for evaluation. All evaluation metrics reported here are computed using the test set.

We also will need to assess our models, as such we need to decide on some metrics. 

## Data Splitting for Evaluation

More specifically, we refer to this tehcnique as **cross-validation holdout**. It is a technique used to evaluate the performance of a machine learning model. It is a method of splitting the available data into three sets: training, validation, and test. The holdout method is one of the simplest way of performing cross-validation. The training set is used to train the model, the validation set is used to evaluate the model and tune its parameters, and the test set is used to evaluate the final performance of the model.

The validation set is particularly important for preventing overfitting. Overfitting occurs when a model is trained too well on the training set and performs poorly on new, unseen data. By using a separate validation set to evaluate the model during training, we can detect overfitting and adjust the model accordingly, before using the test set to evaluate the final performance.

**Note**: a more robust evaluation approach could be to split train and test sets by a reference date, where the train set is composed by all interactions before that date, and the test set are interactions after that date. For the sake of simplicity, we chose the first random approach for this notebook. 

In [63]:
# split data into training and testing
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

NameError: name 'interactions_full_df' is not defined

##  Evaluation Metrics

